# DataLoader

PyTorchでミニバッチ学習を行うときに便利なクラス。

Reference: https://pytorch.org/docs/stable/data.html

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader


---

## `Dataset`

DataLoaderを作るために必要なクラス。  
扱うデータセットとその情報を記述する。

PyTorch側で用意されている基底クラスを継承して作成する。

In [2]:
from torch.utils.data import Dataset
Dataset

torch.utils.data.dataset.Dataset

最低限必要な特殊メソッドが2つある。
- `__len__(self)`: データ数を返す。
- `__getitem__(self, idx: int)`: 指定したインデックスのインスタンス（データ）を返す。

最も簡単な例を実装してみる。

In [3]:
class SimpleDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

実際に適当なデータを入れてみる。

In [4]:
data = torch.tensor([1, 2, 3, 4, 5])
dataset = SimpleDataset(data)
n_data = len(dataset)
print('num of data:', n_data)
for i in range(n_data):
    print(f'index {i}:', dataset[i])

num of data: 5
index 0: tensor(1)
index 1: tensor(2)
index 2: tensor(3)
index 3: tensor(4)
index 4: tensor(5)


教師あり学習の場合は、`__getitem__`で正解も返すようにする。

In [5]:
class SimpleDatasetWithTarget(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

    def __len__(self):
        return len(self.data)

In [6]:
target = torch.tensor([1, 4, 9, 16, 25])
dataset = SimpleDatasetWithTarget(data, target)
for i in range(len(dataset)):
    x, t = dataset[i]
    print(f'index {i}: data={x}, target={t}')

index 0: data=1, target=1
index 1: data=2, target=4
index 2: data=3, target=9
index 3: data=4, target=16
index 4: data=5, target=25



---

## `DataLoader`

`Dataset`を元にミニバッチ学習を行うためのクラス。  
バッチサイズ分のデータを取り出すイテレータ。

In [7]:
from torch.utils.data import DataLoader
DataLoader

torch.utils.data.dataloader.DataLoader

In [8]:
dataset = SimpleDataset(data)
batch_size = 2

dataloader = DataLoader(dataset, batch_size=batch_size)
for x in dataloader:
    print(x)

tensor([1, 2])
tensor([3, 4])
tensor([5])


\[1, 2, 3, 4, 5]というデータからバッチサイズ2で順番にデータを取り出した。  
取り出されたデータは自動的に`torch.Tensor`に変換される。

バッチサイズのデフォルト値は1。

In [9]:
dataloader = DataLoader(dataset)
for x in dataloader:
    print(x)

tensor([1])
tensor([2])
tensor([3])
tensor([4])
tensor([5])


`Dataset`が`__getitem__`でタプルを返す場合は`DataLoader`もタプルを返す。

In [10]:
dataset = SimpleDatasetWithTarget(data, target)
dataloader = DataLoader(dataset)
for batch in dataloader:
    print(batch)

[tensor([1]), tensor([1])]
[tensor([2]), tensor([4])]
[tensor([3]), tensor([9])]
[tensor([4]), tensor([16])]
[tensor([5]), tensor([25])]


（本当は`list`）

In [11]:
type(batch)

list

多次元データの場合も同様に`torch.Tensor`として取り出される。

In [12]:
data = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])
dataset = SimpleDataset(data)
batch_size = 2

dataloader = DataLoader(dataset, batch_size=batch_size)
for x in dataloader:
    print(x)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[ 7,  8,  9],
        [10, 11, 12]])


ただしリストとして与えると少し挙動が変わる。

In [13]:
data = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
]
dataset = SimpleDataset(data)
batch_size = 2

dataloader = DataLoader(dataset, batch_size=batch_size)
for x in dataloader:
    print(x)

[tensor([1, 4]), tensor([2, 5]), tensor([3, 6])]
[tensor([ 7, 10]), tensor([ 8, 11]), tensor([ 9, 12])]


各インスタンスを1次元の配列として見て、位置ごとに`torch.Tensor`にまとめられて取り出される。  
各インスタンスの次元数が揃っていないとエラーになる。

In [14]:
data = [
    [1, 2, 3],
    [4, 5, 6, 0], # 次元数をずらしてみる
    [7, 8, 9],
    [10, 11, 12]
]
dataset = SimpleDataset(data)
batch_size = 2

dataloader = DataLoader(dataset, batch_size=batch_size)
try:
    next(iter(dataloader))
except Exception as e:
    print(e)

each element in list of batch should be of equal size


ここからは引数をいくつか紹介する。

In [15]:
data = torch.tensor([1, 2, 3, 4, 5])

### `shuffle`

`bool`

データの順番をランダムにするかを指定する。

In [16]:
dataset = SimpleDataset(data)
dataloader = DataLoader(dataset, shuffle=True)
for x in dataloader:
    print(x)

tensor([5])
tensor([2])
tensor([3])
tensor([1])
tensor([4])


### `drop_last`

`bool`

データ数がバッチサイズで割り切れない場合に最後のバッチを捨てるかを指定する。

In [17]:
dataset = SimpleDataset(data)
batch_size = 2
dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True)
for x in dataloader:
    print(x)

tensor([1, 2])
tensor([3, 4])


### collate_fn

`Callable`

バッチ内のインスタンスをまとめる関数を指定する。  
インスタンスのリストを入力として受け取る関数。出力はなんでもいい。

自動的に`torch.Tensor`に変換されるのは、実質的にここのデフォルト値として`torch.stack`が使われているからとも解釈できる。

試しに`sum`を指定してみると

In [18]:
dataset = SimpleDataset(data)
batch_size = 2
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=sum)
for x in dataloader:
    print(x)

tensor(3)
tensor(7)
tensor(5)


- `sum([tensor(1), tensor(2)]) = tensor(3)`
- `sum([tensor(3), tensor(4)]) = tensor(7)`
- `sum([tensor(5)]) = tensor(5)`

例えば恒等関数を設定すればバッチサイズ分のインスタンスをそのままリストとして返せる。  
こうすればインスタンスの次元数が異なっていても問題ない。

In [19]:
data = [
    [1, 2, 3],
    [4, 5, 6, 0], # 次元数をずらす
    [7, 8, 9],
    [10, 11, 12]
]
dataset = SimpleDataset(data)
batch_size = 2

dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=lambda x: x)
for x in dataloader:
    print(x)

[[1, 2, 3], [4, 5, 6, 0]]
[[7, 8, 9], [10, 11, 12]]


上手く使えば次元数が異なっていても`torch.Tensor`でも返せる。  
例えば、最も少ない次元数に合わせて切り捨てるとか。

In [20]:
def collate_fn(batch):
    l = min(map(len, batch))
    return torch.tensor([x[:l] for x in batch])

dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
for x in dataloader:
    print(x)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[ 7,  8,  9],
        [10, 11, 12]])
